In [1]:
import numpy as np 
import cv2
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
with_specs = np.load("With_specs.npy")
without_specs = np.load("Without_specs.npy")

In [3]:
with_specs.shape

(200, 50, 50, 3)

In [4]:
without_specs.shape

(200, 50, 50, 3)

In [5]:
with_specs = with_specs.reshape(200,50 * 50 * 3)
without_specs = without_specs.reshape(200,50 * 50 * 3)

In [6]:
with_specs.shape

(200, 7500)

In [7]:
without_specs.shape

(200, 7500)

In [8]:
X = np.r_[with_specs, without_specs]

In [9]:
X.shape

(400, 7500)

In [10]:
# Labeling of Data 
# First 200 with Specs or next 200 without Specs 
# with Specs will labeled as 0 
# without Specs will labeled as 1 
labels = np.zeros(X.shape[0]) 

In [11]:
labels[200:] = 1.0

In [12]:
names = {0 : "SPECS", 1 : 'No SPECS'}

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X, labels,test_size=0.25)


In [14]:
x_train.shape

(300, 7500)

In [15]:
svm = SVC()
svm.fit(x_train,y_train)

SVC()

In [16]:
y_pred = svm.predict(x_test)
accuracy_score(y_test, y_pred)

1.0

In [18]:
haar_data = cv2.CascadeClassifier('spec.xml')
capture = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_DUPLEX
while True :
    flag, img = capture.read()
    if flag : 
        faces = haar_data.detectMultiScale(img)
        for x,y,w,h in faces :
            cv2.rectangle(img, (x,y), (x+w, y+h), (245,224,69), 2)
            face = img[y:y+h , x:x+w, :]
            face = cv2.resize(face,(50,50))
            face = face.reshape(1,-1)
            pred = svm.predict(face)[0]
            n = names[int(pred)]
            cv2.putText(img, n, (x,y), font , 1 , (244,250,250),2)
            #print(n)
        cv2.imshow('Detecting Specs..!',img)
        if cv2.waitKey(2) == 27 :
            break

capture.release()
cv2.destroyAllWindows()